In [6]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import csv
import json
import numpy as np
import pymongo

In [16]:
# Read CSV into dataframe
brew_csv = "../data/beer_reviews.csv"
brewer_df = pd.read_csv(brew_csv)

craft_csv = "../data/craft_beer_fest_data3.csv"
craft_df = pd.read_csv(craft_csv)

# style_csv = "../../../gorbas/beerStyles.csv"
# style_df = pd.read_csv(style_csv)

# rename columns for merging later
craft_df.rename(columns={'Beer':'beer_name','Brewery':'brewery_name'}, inplace=True)

#drop duplicates from this data set
craft_df = craft_df.drop_duplicates(subset="beer_name", keep='first', inplace=False)

# style_df.rename(columns={'type':'Style'}, inplace=True)
brewer_df.head(20)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986.0
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213.0
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215.0
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969.0
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883.0
5,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159.0
6,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159.0
7,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159.0
8,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159.0
9,1075,Caldera Brewing Company,1285632924,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159.0


In [23]:
group_by_beer_name_mean = brewer_df.groupby('beer_name').mean()
group_by_beer_name_mean = group_by_beer_name_mean.reset_index()
group_by_beer_name_mean.head(20)

raw_beer_df = pd.merge(group_by_beer_name_mean, craft_df, on=["beer_name"], how='outer')
# raw_beer_df = raw_beer_df.dropna(how='any').reset_index(drop=True)
raw_beer_df = raw_beer_df.drop(columns=['brewery_id', 'review_time','Table','Ratings','ABV','URL','beer_beerid','Score','Bro Score'])

# print(al_beer_df.columns.values)

avg_scores = list(raw_beer_df["review_overall"])
avg_overall_scores = list(raw_beer_df["rAvg"])
weighted_averages = []

for av, ov in zip(avg_scores, avg_overall_scores):
    weighted_avg = (av+ov)/2
    weighted_averages.append(weighted_avg)

raw_beer_df['Avg Overall Score'] = weighted_averages
al_beer_df = raw_beer_df.drop(columns=['review_overall', 'rAvg'])

# al_beer_df = pd.merge(raw_beer_df, style_df, on=["Style"], how='outer')
# raw_beer_df['group'].value_counts()
# raw_beer_df['group'].isna().sum()

raw_beer_df = raw_beer_df.dropna(how='any').reset_index(drop=True)
raw_beer_df.head(20)


,beer_name,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,brewery_name,Location,rAvg,Style,Avg Overall Score
0,Coco Loco,4.166667,4.0,4.166667,4.166667,4.416667,6.2,NoDa Brewing Company,"Charlotte, NC",4.04,American Porter,4.103333
1,Flower Child IPA,4.000000,3.0,3.500000,3.500000,4.000000,6.5,Cambridge Brewing Company,"Cambridge, MA",4.13,American IPA,4.065000
2,Hefeweizen,4.000000,4.0,3.000000,3.833333,3.833333,5.0,Cambridge Brewing Company,"Cambridge, MA",4.15,Hefeweizen,4.075000


In [13]:
def get_average_scores(column, dfToFilter):
    group_by_current = dfToFilter.groupby(column).mean()

    counts_df = dfToFilter[column].value_counts().to_frame(name=None).reset_index()
    counts_df = counts_df.rename(index=str, columns={"index": column, column: "Quantity"})

    with_counts_df = pd.merge(group_by_current, counts_df, on=[column], how='outer')
    with_counts_df = with_counts_df.rename(index=str, columns={"review_aroma":"Aroma","review_appearance":"Appearance",
                                                                       "review_palate":"Palate","review_taste":"Taste",
                                                                       "beer_abv":"ABV(%)"})
    with_counts_df = with_counts_df.transpose()
    return with_counts_df

def create_mongo(column, groupedDfToFilter):
    final_json = groupedDfToFilter.to_json()
    print(final_json)

    mongo_db = pymongo.MongoClient('localhost', 27017)['Brews']

    db_to_add = mongo_db[column]
    db_to_add.remove()
    db_to_add.insert(json.loads(final_json))

In [15]:
# grouped_df.to_csv("brewery_names_summary.csv", encoding="utf-8", index=False)
# columnsToGroup = ["brewery_name","City","State","Style"]
columnsToGroup = ["brewery_name"]
dfsForMongo = []

for column in columnsToGroup:
    final_grouped_df = get_average_scores(column, al_beer_df)
#     print(final_grouped_df.head())
#     for index, row in final_grouped_df.iterrows():
#         print(row['Quantity'], row['brewery_name'])
    dfsForMongo.append(final_grouped_df)

dfsForMongo

[                                           0
 brewery_name       Cambridge Brewing Company
 Aroma                                      4
 Appearance                                 3
 Palate                               3.83333
 Taste                                3.83333
 ABV(%)                                     5
 Avg Overall Score                      4.075
 Quantity                                   1]

In [28]:
for column,df in zip(columnsToGroup, dfsForMongo):
    create_mongo(column,df)

{"0":{"brewery_name":"Abita Brewing Co.","Aroma":3.2197345693,"Appearance":3.3911331184,"Palate":3.2381851681,"Taste":3.2543182865,"ABV(%)":4.4703030303,"Avg Overall Score":3.3770235491,"Quantity":2},"1":{"brewery_name":"Allagash Brewing Company","Aroma":4.1289077698,"Appearance":4.0518174656,"Palate":4.0689926854,"Taste":4.1913323866,"ABV(%)":7.35,"Avg Overall Score":4.1924360702,"Quantity":2},"2":{"brewery_name":"Anderson Valley Brewing Company","Aroma":3.9233104067,"Appearance":3.9927631579,"Palate":3.9282446172,"Taste":4.055173445,"ABV(%)":6.1633333333,"Avg Overall Score":3.9553513756,"Quantity":2},"3":{"brewery_name":"Avery Brewing Company","Aroma":3.7677636322,"Appearance":3.8260957397,"Palate":3.7102143403,"Taste":3.7374497739,"ABV(%)":7.0306060606,"Avg Overall Score":3.7819358374,"Quantity":3},"4":{"brewery_name":"Ballast Point Brewing Company","Aroma":4.1122522927,"Appearance":4.1714374,"Palate":4.085077333,"Taste":4.1640224218,"ABV(%)":8.2,"Avg Overall Score":4.1276730353,"Qu

C:\Users\cflint\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
C:\Users\cflint\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
